ref: https://towardsdatascience.com/a-tutorial-on-scraping-images-from-the-web-using-beautifulsoup-206a7633e948
https://www.crummy.com/software/BeautifulSoup/bs4/doc.zh/

# Breifing

The scripts 
1. first explore with a instagram link witha target tag (i.e., 'ngautaukok');
2. then from tag webpage, explore hot post, and get location (i.e., 'Lok Wah South Estate')
3. finally explore the location instagram link, and get location's specific latitude and longitude.


# Before everythin, Login Instagram First in the browser!

You can run the following _login()_ script <br>
or login via https://www.instagram.com/accounts/login/ in the browswer.

In [ ]:
# ref: https://github.com/softcoder24/insta_share
from datetime import datetime, timedelta

def login(username='YourUserNameOrEmail', password='YourPassword'):

    link = 'https://www.instagram.com/accounts/login/'
    login_url = 'https://www.instagram.com/accounts/login/ajax/'

    time = int(datetime.now().timestamp())
    response = requests.get(link)
    csrf = response.cookies['csrftoken']
    
    payload = {
        'username': username,
        'enc_password': f'#PWD_INSTAGRAM_BROWSER:0:{time}:{password}',
        'queryParams': {},
        'optIntoOneTap': 'false'
    }

    login_header = {
        "User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest",
        "Referer": "https://www.instagram.com/accounts/login/",
        "x-csrftoken": csrf
    }

    login_response = requests.post(login_url, data=payload, headers=login_header)
    json_data = json.loads(login_response.text)

    if json_data["authenticated"]:
    if True:

        print("login successful")
        cookies = login_response.cookies
        cookie_jar = cookies.get_dict()
        csrf_token = cookie_jar['csrftoken']
        print("csrf_token: ", csrf_token)
        session_id = cookie_jar['sessionid']
        print("session_id: ", session_id)
        
    else:
        print("login failed ", login_response.text)

# -- Example Starts --

In [1]:
from bs4 import BeautifulSoup
import requests
import yaml
import json

# Instagram with tag

In [2]:
url = "https://www.instagram.com/explore/tags/ngautaukok/"
html_page = requests.get(url, allow_redirects=True)
soup = BeautifulSoup(html_page.content, 'html.parser')
# print(soup.prettify())

In [3]:
# ref: https://www.coder.work/article/5046952
import json
body = soup.find('body')
script = body.find('script')
# print(script.string)
page_json = script.string.strip().replace('window._sharedData =', '').replace(';','')
# print(page_json)
data = json.loads(page_json)

In [4]:
import yaml
# print(yaml.dump(data, default_flow_style=False))

In [5]:
print(data['entry_data']['TagPage'][0]['graphql']['hashtag'].keys())

dict_keys(['id', 'name', 'allow_following', 'is_following', 'is_top_media_only', 'profile_pic_url', 'edge_hashtag_to_media', 'edge_hashtag_to_top_posts', 'edge_hashtag_to_content_advisory', 'edge_hashtag_to_related_tags', 'edge_hashtag_to_null_state'])


In [6]:
# Get all posts information
entry_data = data['entry_data']['TagPage'][0]['graphql']['hashtag']
num_post = entry_data['edge_hashtag_to_media']['count']
hashtags = entry_data['edge_hashtag_to_media']['edges']
print('#post with this tag: ', num_post)
print('Retrieved posts: ', len(hashtags))

#post with this tag:  6150
Retrieved posts:  72


In [7]:
# Handle hot posts
entry_data = data['entry_data']['TagPage'][0]['graphql']['hashtag']
hot_posts = entry_data['edge_hashtag_to_top_posts']['edges']
print('# Hot posts:',len(hot_posts))

# Hot posts: 9


In [8]:
import yaml
# print(yaml.dump(hot_posts, default_flow_style=False))

In [9]:
post0 = hot_posts[0]['node']
print(post0.keys())
likes = post0['edge_liked_by']['count']
print('This post get #likes: ', likes)

dict_keys(['__typename', 'id', 'edge_media_to_caption', 'shortcode', 'edge_media_to_comment', 'taken_at_timestamp', 'dimensions', 'display_url', 'edge_liked_by', 'edge_media_preview_like', 'owner', 'thumbnail_src', 'thumbnail_resources', 'is_video', 'accessibility_caption'])
This post get #likes:  165


In [10]:
# Get hot post with MOST likes
max_likes = 0
max_post = hot_posts[0]['node']
max_shortcode = ''
for hashtag in hot_posts:
    post = hashtag['node']
    shortcode = post['shortcode']
    likes = post['edge_liked_by']['count']
    if int(likes) > max_likes:
        max_likes = int(likes)
        max_post = post
        max_shortcode = shortcode
#     print('This post get #likes: ', likes)
print('Max #likes: ', max_likes)
print('Most popular post: ', 'https://www.instagram.com/p/'+max_shortcode)

photo_src = max_post['display_url']
print('Photo url: ', photo_src)

Max #likes:  308
Most popular post:  https://www.instagram.com/p/CHDGYcYhDBi
Photo url:  https://scontent-hkt1-1.cdninstagram.com/v/t51.2885-15/e35/122748670_867961336942173_7993839473005866760_n.jpg?_nc_ht=scontent-hkt1-1.cdninstagram.com&_nc_cat=109&_nc_ohc=MIvQAia5L-gAX8SOE0s&edm=ABZsPhsBAAAA&ccb=7-4&oh=599af8c33a45b0ebece340bf31f6780d&oe=6192167D&_nc_sid=4efc9f


## Explore hot post

In [11]:
url = "https://www.instagram.com/p/CHDGYcYhDBi/"
html_page = requests.get(url, allow_redirects=True)
soup = BeautifulSoup(html_page.content, 'html.parser')
# print(soup.prettify())

In [12]:
script = soup.find('script', type='application/ld+json')
# print(script)
page_json = script.string.strip()
# print(page_json)
data = json.loads(page_json)
# print(data)
print(yaml.dump(data, default_flow_style=False))

'@context': http://schema.org
'@type': ImageObject
author:
  '@type': Person
  alternateName: '@appleyangie'
  mainEntityofPage:
    '@id': https://www.instagram.com/appleyangie/
    '@type': ProfilePage
caption: "Me, myself and I \U0001F300\U0001F300\U0001F300\n\U0001F4F8: @yohjieye \n\
  .\n.\n.\n.\n.\n.\n.\n.\n#hongkong #homekong #discoverhongkong #hkig #explorehk #lokwahestate\
  \ #ngautaukok #insidehongkong #visualhongkong #hkinstagram #everydayhongkong #nikeoffwhite\
  \ #summervibes #photography #citypicz"
comment:
- '@type': Comment
  author:
    '@type': Person
    alternateName: '@appleyangie'
    mainEntityofPage:
      '@id': https://www.instagram.com/appleyangie/
      '@type': ProfilePage
  text: "@jiggawoo \U0001F31D"
- '@type': Comment
  author:
    '@type': Person
    alternateName: '@jay.jaewoo.seo'
    mainEntityofPage:
      '@id': https://www.instagram.com/jay.jaewoo.seo/
      '@type': ProfilePage
  text: "@appleyangie aaaaw it\u2019s ok we are all friends it\u20

In [13]:
try:
    name = data['contentLocation']['name']
    address_url = data['contentLocation']['mainEntityofPage']['@id']
    print("Location in this post: ", name, "\nUrl: ", address_url)
except:
    print("No location in this post.")

Location in this post:  Lok Wah South Estate 
Url:  https://www.instagram.com/explore/locations/303526563885357/lok-wah-south-estate/


# Instagram with location 

In [14]:
url = 'https://www.instagram.com/explore/locations/303526563885357/lok-wah-south-estate/'
html_page = requests.get(url, allow_redirects=True)
soup = BeautifulSoup(html_page.content, 'html.parser')
# print(soup.prettify())

In [16]:
# <meta content="22.3205759" property="place:location:latitude">
# <meta content="114.2229965" property="place:location:longitude">
latitude = soup.find('meta', property="place:location:latitude")
# print(latitude) # not work

In [17]:
metas = soup.find_all('meta')
# print(metas)
latitude = 0.
longitude = 0.
image_urls = []
for meta in metas:   
    if meta.get('property') == "place:location:latitude":
        latitude = float(meta.get('content'))
    if meta.get('property') == "place:location:longitude":
        longitude = float(meta.get('content'))
        
print("Retrieve location (latitude, longitude): (%.4f,%.4f)" %(latitude, longitude))

Retrieve location (latitude, longitude): (22.3206,114.2230)


In [18]:
body = soup.find('body')
script = body.find('script')
# print(script.string)
page_json = script.string.strip().replace('window._sharedData =', '').replace(';','')
# print(page_json)
data = json.loads(page_json)
# print(yaml.dump(data, default_flow_style=False))

In [19]:
entry_data = data['entry_data']['LocationsPage'][0]['graphql']['location']
num_loc_post = entry_data['edge_location_to_media']['count']
print(entry_data.keys())
print('Number of posts in this location: ', num_loc_post)

dict_keys(['id', 'name', 'has_public_page', 'lat', 'lng', 'slug', 'blurb', 'website', 'phone', 'primary_alias_on_fb', 'address_json', 'profile_pic_url', 'edge_location_to_media', 'edge_location_to_top_posts', 'directory'])
Number of posts in this location:  293


In [20]:
hot_posts = entry_data['edge_location_to_top_posts']['edges']
print('Retreived hot posts: ', len(hot_posts))

Retreived hot posts:  48


In [21]:
# Get hot post with MOST likes
max_likes = 0
max_post = hot_posts[0]['node']
max_shortcode = ''
for hashtag in hot_posts:
    post = hashtag['node']
    shortcode = post['shortcode']
    likes = post['edge_liked_by']['count']
    if int(likes) > max_likes:
        max_likes = int(likes)
        max_post = post
        max_shortcode = shortcode
#     print('This post get #likes: ', likes)
print('Max #likes: ', max_likes)
print('Most popular post: ', 'https://www.instagram.com/p/'+max_shortcode)

photo_src = max_post['display_url']
print('Photo url: ', photo_src)

Max #likes:  3989
Most popular post:  https://www.instagram.com/p/CILS2t3A_Ut
Photo url:  https://scontent-hkt1-2.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/128194592_1193867581032667_2838567133719320515_n.jpg?_nc_ht=scontent-hkt1-2.cdninstagram.com&_nc_cat=107&_nc_ohc=s71aj8t0aH8AX8L-rfe&edm=ALLxIhEBAAAA&ccb=7-4&oh=438ff3cca0bc471a71e751ca0b18a64d&oe=6193077C&_nc_sid=b2a057
